In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import os
import numpy as np
import requests
from matplotlib.backends.backend_pdf import PdfPages

## read in the daily values for the entire history of Lee's Ferry and the Grand Canyon

In [ ]:
site_num = {'Lees Ferry':'09380000',"Grand Canyon": '09402500'}
urls = dict()
for ckey,cval in site_num.items():
    dv_url = 'http://waterservices.usgs.gov/nwis/dv/?format=rdb'
    dv_url += '&sites={0}'.format(cval)
    #dv_url += '&startDT=2010-01-01'
    dv_url += '&startDT=1880-01-01'
    #dv_url += '&endDT=2018-01-17'
    dv_url += '&parameterCd=00060'
    print(dv_url)
    urls[ckey] = dv_url

In [ ]:
urls

In [ ]:

for ckey,cval in urls.items():
    print ('reading data for {0}'.format(ckey))
    dv_file = requests.get(cval)

    with open(os.path.join('data','pandas','{0}.dat'.format(ckey)), 'w') as ofp:
        for carp in dv_file:
            ofp.write(carp.decode())

In [ ]:
NWISfilename = os.path.join('data','pandas','Lees Ferry.dat')
reconnoiter = open(NWISfilename, 'r').readlines()
for i in np.arange(60):
    print (reconnoiter[i].rstrip())

In [ ]:
numhash = 0 #let's use the as the counter
for line in reconnoiter:
    if line.startswith('#'):
        numhash +=1
    else:
        break
        
print (numhash)

# Read in the time series

In [ ]:
df_dict = dict()

In [ ]:
for ckey in site_num.keys():
    recon = open(os.path.join('data','pandas','{0}.dat'.format(ckey))).readlines()
    numhash = 0 #let's use the as the counter
    for line in recon:
        if line.startswith('#'):
            numhash +=1
        else:
            break

    print (numhash)
    colnames = recon[numhash].rstrip().split()
    colnames[3] = 'Q'
    
    df_dict[ckey] = pd.read_csv(os.path.join('data','pandas','{0}.dat'.format(ckey)),
                          sep='\t',
                          skiprows = numhash+2,
                          names = colnames,
                          parse_dates = True,
                          index_col = 2)

In [ ]:
df_dict

In [ ]:
df_dict['Grand Canyon'].columns

## let's drop all the columns we don't need 
## NB --> what's up with `inplace=True`?

In [ ]:
for ckey, nwis_df in df_dict.items(): 
    nwis_df.drop((['site_no']),axis=1,inplace=True)

    # we can use a list comprehension
    nwis_df.drop([i for i in nwis_df.columns if i.endswith('cd')], axis=1, inplace=True)

## Let's look at things by water year
First, we can make a couple new columns, one for year, and one for water year.

How can we group by water year? Not a very easy Google Kung Fu exercise at first, but what about "Fiscal Year"?
Google "Pandas group by fiscal year"
http://stackoverflow.com/questions/26341272/using-groupby-on-pandas-dataframe-to-group-by-financial-year

In [ ]:
for ckey, nwis_df in df_dict.items(): 
    #make water year by shifting forward the number of days in Oct., Nov., and Dec.
    # NOTE --> shifting by months is less precise
    nwis_df['water_year'] = nwis_df.index.shift(30+31+31,freq='d').year


## So now we can add columns with some unit conversions

## units are $\frac{ft^3}{s}$

## So let's convert to cubic feet per day which we can later sum up by water year
## $\frac{1 ft^3}{s} \times \frac{60s}{min} \times \frac{60min}{hour} \times \frac{24hours}{day} \rightarrow \frac{ft^3}{day}$

## 1 acre-foot = 43559.9 cubic feet

In [ ]:
for ckey, nwis_df in df_dict.items(): 
    nwis_df['Q_cfd'] = nwis_df.Q * 60 * 60 * 24
    nwis_df['Q_af'] = nwis_df.Q_cfd / 43559.9

In [ ]:
nwis_df.head()

## `agg` is for aggregate - very powerful!

In [ ]:
wateryears = dict()
for ckey, nwis_df in df_dict.items(): 

    wateryears[ckey] = nwis_df.groupby('water_year').agg(['count','mean','sum'])


In [ ]:
wateryears['Lees Ferry'].columns

## Let's explore Lee's Ferry in a bit more detail

## First, any missing data?

In [ ]:
# Note this has a multiple index
plt.figure(figsize=(14,4))
wateryears['Lees Ferry']['Q','count'].plot(kind='bar')


## let's look at statistics to see if there are any missing days prior to 2018 (partial year)?

In [ ]:
wateryears['Lees Ferry'].loc[wateryears['Lees Ferry'].index<2018].describe()

## Nice! 25% are leap years (mean is 365.25), and no years have less than 365 days

## The Colorado River Compact mandates there should be 7.5E6 Acre-feet /year
## of flow at Lee's Ferry. Is that happening?
https://en.wikipedia.org/wiki/Colorado_River_Compact 

https://www.usbr.gov/lc/region/pao/pdfiles/crcompct.pdf

In [ ]:
wateryears['Lees Ferry']['Q_af','sum'].loc[wateryears['Lees Ferry'].index<1963].plot()
plt.plot([1922,1963],[7.5e6,7.5e6])

In [ ]:
wateryears['Lees Ferry']['Q_af','sum'].loc[wateryears['Lees Ferry'].index].plot()
plt.plot([1922,wateryears['Lees Ferry'].index.max()],[7.5e6,7.5e6])

## Let's do some more exploration of the flow over all record

In [ ]:
plt.figure(figsize=(12,8))
df_dict['Lees Ferry'].Q.plot()
plt.tight_layout()
plt.savefig('LeesFerryOnePlot.pdf')

## aggregate by group

In [ ]:
nwis_df = df_dict['Lees Ferry'].copy()

### first we can apply functions to `groupby` grouping

In [ ]:
fig = plt.figure(figsize=(12,4))

mean_Q = nwis_df.Q.groupby(nwis_df.index.year).mean()
lower_CI = mean_Q - 2*nwis_df.Q.groupby(nwis_df.index.year).std()
upper_CI = mean_Q + 2*nwis_df.Q.groupby(nwis_df.index.year).std()
ax = mean_Q.plot(style='b.-')
plt.fill_between(lower_CI.index,lower_CI,upper_CI, color='b',alpha = 0.2)

# can accomplish this in one step using `aggregate`

In [ ]:
Q_agg = nwis_df.Q.groupby(nwis_df.index.year).aggregate([np.min, np.mean, np.std])

In [ ]:
def plot_agg(Q_agg_in):
    Q_agg = Q_agg_in.copy()
    Q_agg.reset_index(drop=True, inplace=True)
    fig = plt.figure(figsize=(12,4))
    lower_CI = Q_agg['mean'] - 2*Q_agg['std']
    upper_CI = Q_agg['mean'] + 2*Q_agg['std']
    ax = Q_agg['mean'].plot(style='b.-')
    plt.fill_between(Q_agg.index,lower_CI,upper_CI, color='b',alpha = 0.2)
plot_agg(Q_agg)

In [ ]:
fig = plt.figure(figsize=(12,4))
nwis_df.Q.groupby(nwis_df.water_year).std().plot(kind='bar',rot=45)

### how hard to change from annual aggregation to monthly?

In [ ]:
Q_agg_month = nwis_df.Q.groupby([nwis_df.index.year, nwis_df.index.month]).aggregate([np.min, np.mean, np.std])
plot_agg(Q_agg_month)

## navigate

In [ ]:
nwis_df.loc[nwis_df.water_year<1963].Q.plot()

In [ ]:
nwis_df.loc[(nwis_df.water_year<1993) & (nwis_df.water_year>=1963)].Q.plot()

## set values

In [ ]:
nwis_df = df_dict['Lees Ferry'].copy()
nwis_df.loc[(nwis_df.index.year<1990) & (nwis_df.index.year>1948), 'Q'] *= 10

In [ ]:
nwis_df.Q.plot()

In [ ]:
# set them back
nwis_df.loc[(nwis_df.index.year<1990) & (nwis_df.index.year>1948), 'Q'] /= 10

## groupby

In [ ]:
plot_fig=True
if plot_fig:
    with PdfPages(os.path.join('data','allyears_LeesFerry.pdf')) as outpdf:
        for cname,cgroup in nwis_df.groupby(nwis_df.index.year):
            print(cname)
            plt.figure()
            cgroup.Q.plot(title=cname)
            plt.ylabel('cfs')
            plt.xlabel('date')
            plt.tight_layout()
            outpdf.savefig()
            plt.close('all')


## now let's look at Lees Ferry and the Grand Canyon together

## we checked out the Lees Ferry record and found no gaps, but what about Grand Canyon?

In [ ]:
# Note this has a multiple inbdex
plt.figure(figsize=(14,4))
wateryears['Grand Canyon']['Q','count'].plot(kind='bar')


## crumbs... looks like some year in the 1990s is short some data

In [ ]:
wateryears['Grand Canyon'].loc[wateryears['Grand Canyon'].index<2018].describe()

In [ ]:
wateryears['Grand Canyon']['Q','count'].loc[wateryears['Grand Canyon']['Q','count']<300].plot(kind='bar')

## so what happens if we try to combine the datasets on a common time index?

## let's work just with 1993-1995, making specific dataframes for now

In [ ]:
dfGC = df_dict['Grand Canyon'].copy()
dfLF = df_dict['Lees Ferry'].copy()



## is the GC data less complete?

In [ ]:
print(len(dfGC))
print(len(dfLF))

## how does this sort out with `groupby` and `aggregate`?

In [ ]:
df_GC_agg = dfGC.groupby(dfGC.index.year).aggregate([np.mean,np.std,'count'])
df_GC_agg['Q','mean'].plot()

In [ ]:
df_GC_agg['Q','count'].plot(kind='bar')

## let's trim down to the years around 1994 to explore how to merge them

In [ ]:
dfGC = dfGC.loc[(dfGC.index.year>=1993) & (dfGC.index.year<=1995)]
dfLF = dfLF.loc[(dfLF.index.year>=1993) & (dfLF.index.year<=1995)]

## one easy way is `pd.concat` -- note difference between `inner` and `outer` join

In [ ]:
df_combined=pd.concat([dfLF.Q,dfGC.Q],axis=1,join='inner')

df_combined.columns = [['Q_LF', 'Q_GC']]
df_combined.dtypes

In [ ]:
df_combined.plot()

In [ ]:
df_combined=pd.concat([dfLF.Q,dfGC.Q],axis=1,join='outer')
df_combined.columns = [['Q_LF', 'Q_GC']]
df_combined.dtypes

In [ ]:
df_combined.plot()

## how can we fill in the missing data?

In [ ]:
df_ffill=df_combined.fillna(method='ffill')
df_ffill.plot()

In [ ]:
df_bfill=df_combined.fillna(method='bfill')
df_bfill.plot()

In [ ]:
df_val = df_combined.fillna(value=20000)
df_val.plot()

## or we can impute from the other series somehow

## first just drop the NaN values for comparison

In [ ]:
df_combined.to_csv('tmp')
df_combined = pd.read_csv('tmp', index_col=0)
df_dropna = df_combined.dropna()

In [ ]:
len(df_dropna)

In [ ]:
df_dropna.head()

In [ ]:
df_dropna['Qrat'] = df_dropna.Q_GC/df_dropna.Q_LF


In [ ]:
df_dropna.head()

## ok, let's find the mean ratio and apply that to fill in missing data

In [ ]:
mean_rat = df_dropna.Qrat.mean()
mean_rat

In [ ]:
df_combined['Q_LF'] = [float(i) for i in df_combined['Q_LF'].values]
df_combined.dtypes

In [ ]:
df_combined.Q_GC=df_combined.Q_GC.fillna(df_combined.Q_LF*mean_rat)
        

In [ ]:
np.unique(np.isnan(df_combined.Q_GC))

In [ ]:
df_combined.plot()

In [ ]:
dfGC_monthly = dfGC.resample('M')

In [ ]:
dfGC_monthly

In [ ]:
dfGC_monthly.mean()